In [1]:
"""
B2B Service Suppliers Yellow Pages Scraper
===========================================
Target: B2B suppliers that need custom web apps for:
- Online ordering portals with customer accounts
- Custom quoting/proof approval workflows
- Recurring order management
- Large catalog management with tiered pricing

Focused Niches:
1. Janitorial/Cleaning Suppliers
2. Industrial Safety Suppliers
3. Promotional Products/Print Shops
4. Uniform/Workwear Distributors

These businesses typically:
- Serve other businesses (B2B)
- Have complex pricing (volume tiers, customer-specific)
- Need recurring order functionality
- Often still use phone/fax/email for orders
- Have budget for custom solutions
"""

import time
import re
import random
import os
import json
import hashlib
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

# ============================================================================
#                              CONFIGURATION
# ============================================================================

# === MODE: Choose how to run ===
# "single"    - Run one search term + one location
# "niche"     - Run one NICHE (all its search terms) across all locations
# "all"       - Run everything (comprehensive)
MODE = "single"

# === FOCUSED B2B NICHES ===
# Each niche has multiple search terms to maximize coverage

NICHES = {
    "janitorial": {
        "label": "Janitorial/Cleaning Supplier",
        "terms": [
            "janitorial-supplies",
            "janitorial-equipment-supplies",
            "cleaning-supplies",
            "cleaning-equipment-supplies",
            "sanitation-supplies",
            "paper-products-wholesale",
            "commercial-cleaning-supplies",
            "floor-care-supplies",
            "restroom-supplies",
        ],
        "pitch": "Online ordering portal for recurring cleaning supply orders"
    },
    "safety": {
        "label": "Industrial Safety Supplier",
        "terms": [
            "safety-equipment-supplies",
            "industrial-safety-equipment",
            "personal-protective-equipment",
            "ppe-supplies",
            "industrial-supplies",
            "welding-supplies",
            "industrial-equipment-supplies",
            "fire-protection-equipment",
            "first-aid-supplies",
        ],
        "pitch": "B2B catalog with compliance docs and account pricing"
    },
    "promo": {
        "label": "Promotional Products/Print",
        "terms": [
            "promotional-products",
            "advertising-specialties",
            "screen-printing",
            "custom-printing",
            "printing-services-commercial",
            "signs",
            "banners",
            "trophies-awards",
            "embroidery",
            "business-forms",
            "custom-t-shirts",
        ],
        "pitch": "Custom quote builder with proof approval workflow"
    },
    "uniforms": {
        "label": "Uniform/Workwear Distributor",
        "terms": [
            "uniforms",
            "uniform-supply",
            "work-clothing",
            "corporate-apparel",
            "industrial-uniforms",
            "medical-scrubs",
            "restaurant-uniforms",
            "embroidery-services",
            "work-boots-shoes",
            "safety-clothing",
        ],
        "pitch": "Company accounts with logo/embroidery options and reordering"
    },
}

# === CURRENT SELECTION ===
# For "single" mode: set both niche and term index
# For "niche" mode: set just the niche key
CURRENT_NICHE = "janitorial"  # Options: janitorial, safety, promo, uniforms
CURRENT_TERM_INDEX = 0        # Which term within the niche (for single mode)
CURRENT_LOCATION_INDEX = 0    # Which location (for single mode)

# === LOCATIONS ===
# Focused on tri-state area + expanded to other industrial regions
# These businesses exist everywhere, not just Manhattan
LOCATIONS = [
    # NYC Boroughs (industrial areas)
    "queens-ny",
    "brooklyn-ny",
    "bronx-ny",
    "staten-island-ny",

    # Queens industrial
    "long-island-city-ny",
    "maspeth-ny",
    "jamaica-ny",
    "college-point-ny",

    # Brooklyn industrial
    "sunset-park-brooklyn-ny",
    "red-hook-brooklyn-ny",
    "east-new-york-brooklyn-ny",

    # Bronx industrial
    "hunts-point-bronx-ny",
    "port-morris-bronx-ny",
    "south-bronx-ny",

    # Long Island (lots of suppliers here)
    "long-island-ny",
    "nassau-county-ny",
    "suffolk-county-ny",
    "hauppauge-ny",
    "farmingdale-ny",
    "hicksville-ny",
    "westbury-ny",

    # Westchester/Hudson Valley
    "westchester-county-ny",
    "yonkers-ny",
    "white-plains-ny",
    "mount-vernon-ny",

    # New Jersey (huge industrial base)
    "newark-nj",
    "jersey-city-nj",
    "elizabeth-nj",
    "edison-nj",
    "paterson-nj",
    "clifton-nj",
    "passaic-nj",
    "union-nj",
    "secaucus-nj",
    "kearny-nj",
    "linden-nj",
    "perth-amboy-nj",
    "new-brunswick-nj",
    "middlesex-county-nj",
    "bergen-county-nj",
    "essex-county-nj",
    "hudson-county-nj",

    # Connecticut
    "stamford-ct",
    "bridgeport-ct",
    "new-haven-ct",
    "hartford-ct",
    "waterbury-ct",
    "norwalk-ct",
    "fairfield-county-ct",
]

# === PAGINATION ===
START_PAGE = 1
MAX_PAGES = 5  # Most searches won't have more than this

# === OUTPUT ===
OUTPUT_DIR = "exports_b2b_suppliers"
PROGRESS_FILE = "scrape_progress.json"

# === SCRAPING SETTINGS ===
FETCH_EMAILS = True
DEBUG = False
HEADLESS = False
MIN_DELAY = 4
MAX_DELAY = 8
PAGE_DELAY = 12
LISTING_DELAY = 3
RESTART_DRIVER_EACH_PAGE = True
MAX_RETRIES = 3

# === USER AGENTS ===
USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.2 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:122.0) Gecko/20100101 Firefox/122.0",
]

# ============================================================================
#                              HELPER FUNCTIONS
# ============================================================================

def ensure_output_dir():
    if not os.path.exists(OUTPUT_DIR):
        os.makedirs(OUTPUT_DIR)
        print(f"Created output directory: {OUTPUT_DIR}")


def get_output_filename(niche_key, search_term, location):
    return os.path.join(OUTPUT_DIR, f"yp_{niche_key}_{location}_{search_term}.xlsx")


def generate_lead_id(company_name, phone):
    key = f"{company_name.lower().strip()}|{phone.strip()}"
    return hashlib.md5(key.encode()).hexdigest()[:12]


def load_existing_leads(filepath):
    if os.path.exists(filepath):
        try:
            df = pd.read_excel(filepath)
            return set(
                generate_lead_id(str(row.get("Company Name", "")), str(row.get("Phone Number", "")))
                for _, row in df.iterrows()
            )
        except:
            return set()
    return set()


def load_all_existing_lead_ids():
    all_ids = set()
    if os.path.exists(OUTPUT_DIR):
        for filename in os.listdir(OUTPUT_DIR):
            if filename.endswith(".xlsx"):
                filepath = os.path.join(OUTPUT_DIR, filename)
                all_ids.update(load_existing_leads(filepath))
    print(f"Loaded {len(all_ids)} existing lead IDs for deduplication")
    return all_ids


def save_progress(niche, term_idx, location_idx, status="in_progress"):
    progress = {
        "niche": niche,
        "term_index": term_idx,
        "location_index": location_idx,
        "status": status,
        "timestamp": datetime.now().isoformat()
    }
    with open(PROGRESS_FILE, "w") as f:
        json.dump(progress, f, indent=2)


def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as f:
            return json.load(f)
    return None


def create_driver():
    options = Options()

    if HEADLESS:
        options.add_argument("--headless=new")

    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")

    user_agent = random.choice(USER_AGENTS)
    options.add_argument(f"user-agent={user_agent}")

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)
    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")

    return driver


def random_delay(min_sec=None, max_sec=None):
    min_sec = min_sec or MIN_DELAY
    max_sec = max_sec or MAX_DELAY
    delay = random.uniform(min_sec, max_sec)
    time.sleep(delay)
    return delay


# ============================================================================
#                           EMAIL EXTRACTION
# ============================================================================

EMAIL_BLACKLIST = [
    'example.com', 'domain.com', 'email.com', 'yoursite', 'yourdomain',
    'sentry.io', 'schema.org', 'json', 'wixpress', 'wix.com',
    'googleapis', 'google.com', 'facebook', 'twitter', 'instagram',
    '.png', '.jpg', '.gif', '.svg', '.css', '.js',
    'yellowpages', 'yp.com', 'placeholder', 'test.com',
    'wordpress', 'squarespace', 'shopify', 'godaddy', 'wufoo'
]


def is_valid_email(email):
    if not email or '@' not in email:
        return False
    email_lower = email.lower()
    return not any(x in email_lower for x in EMAIL_BLACKLIST)


def extract_email_from_website(driver, website_url, timeout=15):
    if not website_url or website_url == "N/A":
        return ""

    try:
        if not website_url.startswith("http"):
            website_url = "https://" + website_url

        random_delay(1, 2)
        driver.set_page_load_timeout(timeout)

        try:
            driver.get(website_url)
        except:
            return ""

        time.sleep(2)
        page_source = driver.page_source

        if any(x in driver.title.lower() for x in ["404", "not found", "error", "denied"]):
            return ""

        # Method 1: Mailto links
        mailto_match = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', page_source, re.IGNORECASE)
        if mailto_match:
            email = mailto_match.group(1).strip()
            if is_valid_email(email):
                return email

        # Method 2: Email patterns
        email_matches = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', page_source)
        for email in email_matches:
            if is_valid_email(email):
                return email

        # Method 3: Contact pages
        base_url = website_url.rstrip('/')
        contact_paths = ['/contact', '/contact-us', '/about', '/about-us', '/contactus']

        for path in contact_paths:
            try:
                driver.get(base_url + path)
                time.sleep(1.5)
                contact_source = driver.page_source

                mailto_match = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', contact_source, re.IGNORECASE)
                if mailto_match:
                    email = mailto_match.group(1).strip()
                    if is_valid_email(email):
                        return email

                email_matches = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', contact_source)
                for email in email_matches:
                    if is_valid_email(email):
                        return email
            except:
                continue

    except Exception as e:
        if DEBUG:
            print(f" [website error: {e}]", end="")

    return ""


def extract_email_from_detail(driver, detail_url, website_url="", debug_save=False):
    try:
        random_delay(LISTING_DELAY, LISTING_DELAY + 2)
        driver.get(detail_url)

        try:
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, ".business-info, .sales-info, #main-content, #cf-wrapper"))
            )
        except:
            pass

        time.sleep(2)
        page_source = driver.page_source

        is_blocked = (
            "you have been blocked" in page_source.lower() or
            ("cloudflare" in page_source.lower() and "ray id" in page_source.lower())
        )

        if is_blocked:
            if website_url:
                print(" [blocked, trying website]", end="")
                email = extract_email_from_website(driver, website_url)
                if email:
                    return email
            return "__BLOCKED__"

        if debug_save:
            with open(os.path.join(OUTPUT_DIR, "debug_page.html"), "w", encoding="utf-8") as f:
                f.write(page_source)

        driver.execute_script("window.scrollTo(0, 800);")
        time.sleep(1)
        page_source = driver.page_source

        # Method 1: Mailto in source
        mailto_match = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', page_source, re.IGNORECASE)
        if mailto_match:
            email = mailto_match.group(1).strip()
            if is_valid_email(email):
                return email

        # Method 2: Email link elements
        try:
            email_elements = driver.find_elements(By.CSS_SELECTOR, "a.email-business, a[class*='email']")
            for el in email_elements:
                href = el.get_attribute("href") or ""
                if "mailto:" in href:
                    email = href.replace("mailto:", "").split("?")[0].strip()
                    if is_valid_email(email):
                        return email
        except:
            pass

        # Method 3: Any mailto
        try:
            mailto_links = driver.find_elements(By.CSS_SELECTOR, "a[href*='mailto:']")
            for link in mailto_links:
                href = link.get_attribute("href") or ""
                if "mailto:" in href:
                    email = href.replace("mailto:", "").split("?")[0].strip()
                    if is_valid_email(email):
                        return email
        except:
            pass

        # Method 4: BeautifulSoup
        soup = BeautifulSoup(page_source, "html.parser")
        for link in soup.find_all("a", href=True):
            href = link["href"]
            if "mailto:" in href:
                email = href.replace("mailto:", "").split("?")[0].strip()
                if is_valid_email(email):
                    return email

        # Method 5: Regex
        email_matches = re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', page_source)
        for email in email_matches:
            if is_valid_email(email):
                return email

        # Fallback: Company website
        if website_url:
            print(" [trying website]", end="")
            email = extract_email_from_website(driver, website_url)
            if email:
                return email

    except Exception as e:
        if DEBUG:
            print(f" [error: {e}]", end="")

    return ""


# ============================================================================
#                           LISTING PARSING
# ============================================================================

def parse_listing(listing, niche_label):
    try:
        name_el = listing.select_one(".business-name span")
        if not name_el:
            name_el = listing.select_one(".business-name")
        company = name_el.text.strip() if name_el else ""

        if not company:
            return None

        phone_el = listing.select_one(".phones")
        phone = phone_el.text.strip() if phone_el else ""

        street = listing.select_one(".street-address")
        locality = listing.select_one(".locality")
        address = " ".join(filter(None, [
            street.text.strip() if street else "",
            locality.text.strip() if locality else ""
        ]))

        website_el = listing.select_one(".track-visit-website")
        website = website_el["href"] if website_el else ""

        detail_el = listing.select_one(".business-name")
        detail_link = ""
        if detail_el and detail_el.get("href"):
            detail_link = "https://www.yellowpages.com" + detail_el["href"]

        categories_el = listing.select_one(".categories")
        categories = categories_el.text.strip() if categories_el else ""

        # Check if they have a website (important for qualifying leads)
        has_website = "Yes" if website else "No"

        return {
            "#": None,
            "Company Name": company,
            "Niche": niche_label,
            "Category": categories,
            "Has Website": has_website,
            "Contact Name": "",
            "Email Address": "",
            "Phone Number": phone,
            "Website URL": website,
            "Address": address,
            "Date Added": datetime.now().strftime("%m/%d/%y"),
            "Date Contacted": "",
            "Source": detail_link,
            "Notes": "",
            "Status": "",
            "_lead_id": generate_lead_id(company, phone)
        }
    except Exception as e:
        if DEBUG:
            print(f"  Parse error: {e}")
        return None


def get_listings_from_page(driver, niche_label):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)

    try:
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, ".result, .search-results"))
        )
    except:
        return []

    listings = driver.find_elements(By.CSS_SELECTOR, ".result")
    page_data = []

    for listing in listings:
        try:
            html = listing.get_attribute("outerHTML")
            soup = BeautifulSoup(html, "html.parser")
            parsed = parse_listing(soup, niche_label)
            if parsed:
                page_data.append(parsed)
        except:
            continue

    return page_data


# ============================================================================
#                           EXCEL OUTPUT
# ============================================================================

def add_checkboxes(filepath):
    try:
        wb = load_workbook(filepath)
        ws = wb.active

        # Status dropdown
        status_validation = DataValidation(
            type="list",
            formula1='"Not Contacted,Contacted,Interested,Not Interested,Closed Won,Closed Lost"',
            allow_blank=True
        )
        ws.add_data_validation(status_validation)

        headers = {cell.value: cell.column for cell in ws[1]}

        if "Status" in headers:
            col_idx = headers["Status"]
            for row in range(2, ws.max_row + 1):
                cell = ws.cell(row=row, column=col_idx)
                if not cell.value:
                    cell.value = "Not Contacted"
                status_validation.add(cell)

        wb.save(filepath)
    except Exception as e:
        print(f"  Warning: Could not add dropdowns: {e}")


def save_leads_to_excel(leads, filepath):
    if not leads:
        return

    clean_leads = []
    for lead in leads:
        clean_lead = {k: v for k, v in lead.items() if not k.startswith("_")}
        clean_leads.append(clean_lead)

    for i, lead in enumerate(clean_leads, 1):
        lead["#"] = i

    df = pd.DataFrame(clean_leads)
    df.to_excel(filepath, index=False)
    add_checkboxes(filepath)


# ============================================================================
#                           MAIN SCRAPER
# ============================================================================

def scrape_search(niche_key, search_term, niche_label, location, existing_ids=None):
    existing_ids = existing_ids or set()
    base_url = f"https://www.yellowpages.com/{location}/{search_term}"
    output_file = get_output_filename(niche_key, search_term, location)

    # Load existing
    existing_file_leads = []
    if os.path.exists(output_file):
        try:
            df = pd.read_excel(output_file)
            existing_file_leads = df.to_dict('records')
            for lead in existing_file_leads:
                lead["_lead_id"] = generate_lead_id(
                    str(lead.get("Company Name", "")),
                    str(lead.get("Phone Number", ""))
                )
        except:
            pass

    print(f"\n{'='*70}")
    print(f"SCRAPING: {niche_label}")
    print(f"Search Term: {search_term}")
    print(f"Location: {location}")
    print(f"URL: {base_url}")
    print(f"Existing in file: {len(existing_file_leads)}")
    print(f"{'='*70}\n")

    driver = create_driver()
    all_leads = list(existing_file_leads)
    local_ids = {lead["_lead_id"] for lead in all_leads}
    new_leads_count = 0
    blocked_count = 0

    try:
        for page in range(START_PAGE, MAX_PAGES + 1):
            url = base_url if page == 1 else f"{base_url}?page={page}"

            print(f"[Page {page}] Loading...")

            for attempt in range(MAX_RETRIES):
                try:
                    driver.get(url)
                    time.sleep(2)
                    break
                except Exception as e:
                    if attempt < MAX_RETRIES - 1:
                        print(f"  Retry {attempt + 1}...")
                        time.sleep(5)
                    else:
                        print(f"  Failed: {e}")
                        continue

            page_listings = get_listings_from_page(driver, niche_label)

            if not page_listings:
                print(f"  No listings - end of results")
                break

            # Filter dupes
            new_listings = []
            for listing in page_listings:
                lead_id = listing["_lead_id"]
                if lead_id not in existing_ids and lead_id not in local_ids:
                    new_listings.append(listing)
                    local_ids.add(lead_id)

            print(f"  Found {len(page_listings)} listings, {len(new_listings)} new")

            if not new_listings:
                print(f"  All duplicates - skipping")
                if page < MAX_PAGES:
                    time.sleep(random.uniform(PAGE_DELAY/2, PAGE_DELAY))
                continue

            # Fetch emails
            if FETCH_EMAILS:
                emails_found = 0
                for i, lead in enumerate(new_listings):
                    company_short = lead['Company Name'][:38].ljust(38)
                    print(f"  [{i+1:2}/{len(new_listings)}] {company_short}", end="", flush=True)

                    email = extract_email_from_detail(
                        driver,
                        lead["Source"],
                        website_url=lead.get("Website URL", ""),
                        debug_save=(DEBUG and page == 1 and i == 0)
                    )

                    if email == "__BLOCKED__":
                        print(f" -> BLOCKED")
                        blocked_count += 1
                        if blocked_count >= 5:
                            print("\n  Too many blocks - restarting...")
                            try:
                                driver.quit()
                            except:
                                pass
                            time.sleep(10)
                            driver = create_driver()
                            blocked_count = 0
                    elif email:
                        lead["Email Address"] = email
                        emails_found += 1
                        print(f" -> {email}")
                    else:
                        print(f" -> (no email)")

                print(f"\n  Page {page}: {emails_found}/{len(new_listings)} emails")

            all_leads.extend(new_listings)
            new_leads_count += len(new_listings)

            # Save progress
            save_leads_to_excel(all_leads, output_file)
            print(f"  Saved {len(all_leads)} leads to {output_file}")

            if page < MAX_PAGES:
                if RESTART_DRIVER_EACH_PAGE:
                    print(f"  Restarting browser...")
                    try:
                        driver.quit()
                    except:
                        pass
                    time.sleep(3)
                    driver = create_driver()

                delay = random.uniform(PAGE_DELAY, PAGE_DELAY + 5)
                print(f"  Waiting {delay:.1f}s...\n")
                time.sleep(delay)

    except Exception as e:
        print(f"\nError: {e}")
        if all_leads:
            save_leads_to_excel(all_leads, output_file)

    finally:
        try:
            driver.quit()
        except:
            pass

    email_count = sum(1 for lead in all_leads if lead.get("Email Address"))
    website_count = sum(1 for lead in all_leads if lead.get("Has Website") == "Yes")

    print(f"\n{'='*70}")
    print(f"COMPLETED: {search_term} in {location}")
    print(f"New leads: {new_leads_count}")
    print(f"Total in file: {len(all_leads)}")
    print(f"With emails: {email_count}")
    print(f"With websites: {website_count}")
    print(f"{'='*70}")

    return all_leads, new_leads_count


# ============================================================================
#                           RUN MODES
# ============================================================================

def run_single_search():
    """Run a single search term + location"""
    ensure_output_dir()

    niche = NICHES[CURRENT_NICHE]
    term = niche["terms"][CURRENT_TERM_INDEX]
    location = LOCATIONS[CURRENT_LOCATION_INDEX]

    print(f"\nMODE: Single Search")
    print(f"Niche: {niche['label']}")
    print(f"Term: {term}")
    print(f"Location: {location}")
    print(f"Pitch: {niche['pitch']}")

    existing_ids = load_all_existing_lead_ids()
    scrape_search(CURRENT_NICHE, term, niche["label"], location, existing_ids)


def run_niche_search():
    """Run all terms for one niche across all locations"""
    ensure_output_dir()

    niche_key = CURRENT_NICHE
    niche = NICHES[niche_key]

    print(f"\nMODE: Full Niche Search")
    print(f"Niche: {niche['label']}")
    print(f"Terms: {len(niche['terms'])}")
    print(f"Locations: {len(LOCATIONS)}")
    print(f"Total combinations: {len(niche['terms']) * len(LOCATIONS)}")
    print(f"Pitch: {niche['pitch']}")

    total_new = 0
    existing_ids = load_all_existing_lead_ids()
    combo = 0
    total = len(niche['terms']) * len(LOCATIONS)

    for ti, term in enumerate(niche['terms']):
        for li, location in enumerate(LOCATIONS):
            combo += 1
            print(f"\n>>> [{combo}/{total}] {term} @ {location}")
            save_progress(niche_key, ti, li)

            _, new_count = scrape_search(niche_key, term, niche["label"], location, existing_ids)
            total_new += new_count
            existing_ids = load_all_existing_lead_ids()

            if combo < total:
                delay = random.uniform(20, 40)
                print(f"\nWaiting {delay:.0f}s...\n")
                time.sleep(delay)

    save_progress(niche_key, len(niche['terms'])-1, len(LOCATIONS)-1, "completed")
    print(f"\n{'='*70}")
    print(f"NICHE COMPLETE: {niche['label']}")
    print(f"Total new leads: {total_new}")
    print(f"{'='*70}")


def run_all_niches():
    """Run everything"""
    ensure_output_dir()

    total_combos = sum(len(n['terms']) * len(LOCATIONS) for n in NICHES.values())

    print(f"\nMODE: Full Scrape (all niches)")
    print(f"Niches: {len(NICHES)}")
    print(f"Total combinations: {total_combos}")

    total_new = 0
    existing_ids = load_all_existing_lead_ids()
    combo = 0

    for niche_key, niche in NICHES.items():
        for ti, term in enumerate(niche['terms']):
            for li, location in enumerate(LOCATIONS):
                combo += 1
                print(f"\n>>> [{combo}/{total_combos}] {niche['label']}: {term} @ {location}")
                save_progress(niche_key, ti, li)

                _, new_count = scrape_search(niche_key, term, niche["label"], location, existing_ids)
                total_new += new_count
                existing_ids = load_all_existing_lead_ids()

                if combo < total_combos:
                    delay = random.uniform(20, 40)
                    time.sleep(delay)

    print(f"\n{'='*70}")
    print(f"ALL NICHES COMPLETE!")
    print(f"Total new leads: {total_new}")
    print(f"{'='*70}")


# ============================================================================
#                           UTILITIES
# ============================================================================

def merge_all_files():
    """Merge all files into master list"""
    import glob
    ensure_output_dir()

    files = glob.glob(os.path.join(OUTPUT_DIR, "yp_*.xlsx"))
    files = [f for f in files if "MERGED" not in f and "EMAILS" not in f]

    if not files:
        print("No files to merge!")
        return None

    print(f"Merging {len(files)} files...")

    all_leads = []
    for f in files:
        try:
            df = pd.read_excel(f)
            all_leads.extend(df.to_dict('records'))
        except Exception as e:
            print(f"  Error reading {f}: {e}")

    if not all_leads:
        print("No leads found!")
        return None

    # Dedupe
    seen = set()
    unique = []
    for lead in all_leads:
        key = generate_lead_id(str(lead.get("Company Name", "")), str(lead.get("Phone Number", "")))
        if key not in seen:
            seen.add(key)
            unique.append(lead)

    for i, lead in enumerate(unique, 1):
        lead["#"] = i

    output_path = os.path.join(OUTPUT_DIR, "ALL_LEADS_MERGED.xlsx")
    df = pd.DataFrame(unique)
    df.to_excel(output_path, index=False)
    add_checkboxes(output_path)

    email_count = sum(1 for l in unique if l.get("Email Address"))
    website_count = sum(1 for l in unique if l.get("Has Website") == "Yes")

    print(f"\n{'='*70}")
    print(f"MERGE COMPLETE!")
    print(f"Files merged: {len(files)}")
    print(f"Total unique leads: {len(unique)}")
    print(f"With emails: {email_count}")
    print(f"With websites: {website_count}")
    print(f"Saved to: {output_path}")
    print(f"{'='*70}")

    return df


def export_hot_leads():
    """Export leads that have BOTH email AND website (hottest leads)"""
    merged_path = os.path.join(OUTPUT_DIR, "ALL_LEADS_MERGED.xlsx")

    if not os.path.exists(merged_path):
        print("Run merge_all_files() first!")
        return None

    df = pd.read_excel(merged_path)

    # Hot leads: have email
    hot = df[df["Email Address"].notna() & (df["Email Address"] != "")]
    hot = hot.copy()
    hot["#"] = range(1, len(hot) + 1)

    output_path = os.path.join(OUTPUT_DIR, "HOT_LEADS_WITH_EMAILS.xlsx")
    hot.to_excel(output_path, index=False)
    add_checkboxes(output_path)

    print(f"Exported {len(hot)} hot leads (with emails) to: {output_path}")

    # Also export leads WITHOUT websites (need your services most!)
    no_website = df[df["Has Website"] == "No"]
    no_website = no_website.copy()
    no_website["#"] = range(1, len(no_website) + 1)

    output_path2 = os.path.join(OUTPUT_DIR, "LEADS_NO_WEBSITE.xlsx")
    no_website.to_excel(output_path2, index=False)
    add_checkboxes(output_path2)

    print(f"Exported {len(no_website)} leads WITHOUT websites to: {output_path2}")

    return hot


def export_by_niche():
    """Export by niche type"""
    merged_path = os.path.join(OUTPUT_DIR, "ALL_LEADS_MERGED.xlsx")

    if not os.path.exists(merged_path):
        print("Run merge_all_files() first!")
        return

    df = pd.read_excel(merged_path)

    for niche in df["Niche"].unique():
        niche_df = df[df["Niche"] == niche].copy()
        niche_df["#"] = range(1, len(niche_df) + 1)

        safe_name = niche.replace("/", "-").replace(" ", "_").lower()
        output_path = os.path.join(OUTPUT_DIR, f"LEADS_{safe_name}.xlsx")
        niche_df.to_excel(output_path, index=False)
        add_checkboxes(output_path)

        email_ct = niche_df["Email Address"].notna().sum()
        print(f"  {niche}: {len(niche_df)} leads ({email_ct} with email) -> {output_path}")


def print_stats():
    """Show statistics"""
    merged_path = os.path.join(OUTPUT_DIR, "ALL_LEADS_MERGED.xlsx")

    if not os.path.exists(merged_path):
        print("Run merge_all_files() first!")
        return

    df = pd.read_excel(merged_path)

    print(f"\n{'='*70}")
    print("LEAD STATISTICS")
    print(f"{'='*70}")
    print(f"Total leads: {len(df)}")
    print(f"With emails: {df['Email Address'].notna().sum()}")
    print(f"With websites: {(df['Has Website'] == 'Yes').sum()}")
    print(f"WITHOUT websites: {(df['Has Website'] == 'No').sum()} <- BEST PROSPECTS!")
    print(f"\nBy Niche:")
    for niche in df["Niche"].unique():
        niche_df = df[df["Niche"] == niche]
        emails = niche_df["Email Address"].notna().sum()
        no_web = (niche_df["Has Website"] == "No").sum()
        print(f"  {niche}: {len(niche_df)} total, {emails} emails, {no_web} no website")
    print(f"{'='*70}")


def print_niches():
    """Show available niches and their search terms"""
    print(f"\n{'='*70}")
    print("AVAILABLE NICHES")
    print(f"{'='*70}")
    for key, niche in NICHES.items():
        print(f"\n{key}: {niche['label']}")
        print(f"  Pitch: {niche['pitch']}")
        print(f"  Terms ({len(niche['terms'])}):")
        for i, term in enumerate(niche['terms']):
            print(f"    [{i}] {term}")
    print(f"\n{'='*70}")


# ============================================================================
#                           MAIN
# ============================================================================

if __name__ == "__main__":
    print(f"\n{'='*70}")
    print("B2B SUPPLIER LEAD SCRAPER")
    print("Focused: Janitorial | Safety | Promo | Uniforms")
    print(f"{'='*70}")
    print(f"Mode: {MODE}")
    print(f"Current Niche: {CURRENT_NICHE}")
    print(f"Output: {OUTPUT_DIR}")
    print(f"{'='*70}\n")

    if MODE == "single":
        run_single_search()
    elif MODE == "niche":
        run_niche_search()
    elif MODE == "all":
        run_all_niches()
    else:
        print(f"Unknown mode: {MODE}")
        print("Valid: single, niche, all")



B2B SUPPLIER LEAD SCRAPER
Focused: Janitorial | Safety | Promo | Uniforms
Mode: single
Current Niche: janitorial
Output: exports_b2b_suppliers

Created output directory: exports_b2b_suppliers

MODE: Single Search
Niche: Janitorial/Cleaning Supplier
Term: janitorial-supplies
Location: queens-ny
Pitch: Online ordering portal for recurring cleaning supply orders
Loaded 0 existing lead IDs for deduplication

SCRAPING: Janitorial/Cleaning Supplier
Search Term: janitorial-supplies
Location: queens-ny
URL: https://www.yellowpages.com/queens-ny/janitorial-supplies
Existing in file: 0

[Page 1] Loading...
  Found 30 listings, 29 new
  [ 1/29] CleanZone Inc                          -> BLOCKED
  [ 2/29] Sds Janitorial Supplies                -> BLOCKED
  [ 3/29] Kitro Supplies Inc                     [blocked, trying website] -> kitrosupplies@yahoo.com
  [ 4/29] ALL CITY SUPPLY                        -> BLOCKED
  [ 5/29] National Janitorial Supplies           -> BLOCKED
  [ 6/29] MBA Supply Comp

In [1]:
# Promotional Products/Print Shops Scraper
# Pitch: "Custom quote builder with proof approval workflow"
# FILTERS: Only businesses with real websites (no blank, localsearch, yellowpages URLs)
# Copy this entire file into a Jupyter cell and run

import time, re, random, os, json, hashlib
from datetime import datetime
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import pandas as pd
from openpyxl import load_workbook
from openpyxl.worksheet.datavalidation import DataValidation

# ============== CONFIG ==============
NICHE_KEY = "promo"
NICHE_LABEL = "Promotional Products/Print"
SEARCH_TERMS = [
    "promotional-products",
    "advertising-specialties",
    "screen-printing",
    "custom-printing",
    "printing-services-commercial",
    "signs",
    "banners",
    "trophies-awards",
    "embroidery",
    "business-forms",
    "custom-t-shirts",
]

LOCATIONS = [
    "queens-ny", "brooklyn-ny", "bronx-ny", "staten-island-ny",
    "long-island-city-ny", "maspeth-ny", "jamaica-ny", "college-point-ny",
    "sunset-park-brooklyn-ny", "red-hook-brooklyn-ny", "east-new-york-brooklyn-ny",
    "hunts-point-bronx-ny", "port-morris-bronx-ny", "south-bronx-ny",
    "long-island-ny", "nassau-county-ny", "suffolk-county-ny", "hauppauge-ny",
    "farmingdale-ny", "hicksville-ny", "westbury-ny",
    "westchester-county-ny", "yonkers-ny", "white-plains-ny",
    "newark-nj", "jersey-city-nj", "elizabeth-nj", "edison-nj",
    "paterson-nj", "clifton-nj", "union-nj", "secaucus-nj", "kearny-nj",
    "linden-nj", "perth-amboy-nj", "new-brunswick-nj",
    "middlesex-county-nj", "bergen-county-nj", "essex-county-nj",
    "stamford-ct", "bridgeport-ct", "new-haven-ct", "hartford-ct", "norwalk-ct",
]

CURRENT_TERM_INDEX = 0
CURRENT_LOCATION_INDEX = 0
RUN_ALL = True

OUTPUT_DIR = "exports_promo"
MAX_PAGES = 5
FETCH_EMAILS = True
HEADLESS = False
MIN_DELAY, MAX_DELAY = 4, 8
PAGE_DELAY, LISTING_DELAY = 12, 3

USER_AGENTS = [
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 Chrome/121.0.0.0 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 Version/17.2 Safari/605.1.15",
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:122.0) Gecko/20100101 Firefox/122.0",
]
EMAIL_BLACKLIST = ['example.com','domain.com','sentry.io','schema.org','wixpress','googleapis','yellowpages','.png','.jpg','.css','.js']

# ============== WEBSITE FILTER ==============
def is_valid_website(url):
    """Filter out blank, localsearch, and yellowpages URLs - we only want real business websites"""
    if not url or not url.strip():
        return False
    url_lower = url.lower().strip()
    invalid_patterns = [
        'localsearch.com',
        'yellowpages.com',
        'yp.com',
        'superpages.com',
        'whitepages.com',
        'manta.com',
        'yelp.com',
    ]
    return not any(pattern in url_lower for pattern in invalid_patterns)

# ============== HELPERS ==============
def ensure_dir():
    if not os.path.exists(OUTPUT_DIR): os.makedirs(OUTPUT_DIR)

def gen_id(name, phone):
    return hashlib.md5(f"{str(name).lower().strip()}|{str(phone).strip()}".encode()).hexdigest()[:12]

def load_all_ids():
    ids = set()
    if os.path.exists(OUTPUT_DIR):
        for f in os.listdir(OUTPUT_DIR):
            if f.endswith(".xlsx"):
                try:
                    df = pd.read_excel(os.path.join(OUTPUT_DIR, f))
                    ids.update(gen_id(r.get("Company Name",""), r.get("Phone Number","")) for _,r in df.iterrows())
                except: pass
    return ids

def create_driver():
    opts = Options()
    if HEADLESS: opts.add_argument("--headless=new")
    opts.add_argument("--no-sandbox"); opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--disable-gpu"); opts.add_argument("--window-size=1920,1080")
    opts.add_argument("--disable-blink-features=AutomationControlled")
    opts.add_argument(f"user-agent={random.choice(USER_AGENTS)}")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option('useAutomationExtension', False)
    d = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=opts)
    d.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined})")
    return d

def valid_email(e):
    return e and '@' in e and not any(x in e.lower() for x in EMAIL_BLACKLIST)

def get_email_from_site(driver, url):
    if not url or not is_valid_website(url): return ""
    try:
        if not url.startswith("http"): url = "https://" + url
        driver.set_page_load_timeout(15)
        try: driver.get(url)
        except: return ""
        time.sleep(2); src = driver.page_source
        m = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', src, re.IGNORECASE)
        if m and valid_email(m.group(1)): return m.group(1).strip()
        for e in re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', src):
            if valid_email(e): return e
        for p in ['/contact','/contact-us','/about']:
            try:
                driver.get(url.rstrip('/') + p); time.sleep(1.5); src = driver.page_source
                m = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', src, re.IGNORECASE)
                if m and valid_email(m.group(1)): return m.group(1).strip()
                for e in re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', src):
                    if valid_email(e): return e
            except: pass
    except: pass
    return ""

def get_email(driver, detail_url, website=""):
    try:
        time.sleep(random.uniform(LISTING_DELAY, LISTING_DELAY+2))
        driver.get(detail_url); time.sleep(2); src = driver.page_source
        if "blocked" in src.lower() or ("cloudflare" in src.lower() and "ray id" in src.lower()):
            if website and is_valid_website(website): return get_email_from_site(driver, website)
            return "__BLOCKED__"
        driver.execute_script("window.scrollTo(0,800)"); time.sleep(1); src = driver.page_source
        m = re.search(r'href=["\']mailto:([^"\'<>?\s]+)', src, re.IGNORECASE)
        if m and valid_email(m.group(1)): return m.group(1).strip()
        for e in re.findall(r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}', src):
            if valid_email(e): return e
        if website and is_valid_website(website): return get_email_from_site(driver, website)
    except: pass
    return ""

def parse_listing(lst):
    """Parse listing - returns None if no valid website (filtered out)"""
    try:
        nm = lst.select_one(".business-name span") or lst.select_one(".business-name")
        name = nm.text.strip() if nm else ""
        if not name: return None
        web_el = lst.select_one(".track-visit-website")
        web = web_el["href"] if web_el else ""
        if not is_valid_website(web): return None
        ph = lst.select_one(".phones"); phone = ph.text.strip() if ph else ""
        st = lst.select_one(".street-address"); loc = lst.select_one(".locality")
        addr = " ".join(filter(None, [st.text.strip() if st else "", loc.text.strip() if loc else ""]))
        det = lst.select_one(".business-name"); link = "https://www.yellowpages.com" + det["href"] if det and det.get("href") else ""
        cat = lst.select_one(".categories"); cats = cat.text.strip() if cat else ""
        return {"#":None,"Company Name":name,"Niche":NICHE_LABEL,"Category":cats,
                "Email":"","Phone":phone,"Website":web,"Address":addr,"Date Added":datetime.now().strftime("%m/%d/%y"),
                "Source":link,"Status":"","Notes":"","_id":gen_id(name,phone)}
    except: return None

def get_page_listings(driver):
    driver.execute_script("window.scrollTo(0,document.body.scrollHeight)"); time.sleep(2)
    try: WebDriverWait(driver,10).until(EC.presence_of_element_located((By.CSS_SELECTOR,".result")))
    except: return []
    data = []
    for el in driver.find_elements(By.CSS_SELECTOR, ".result"):
        try:
            p = parse_listing(BeautifulSoup(el.get_attribute("outerHTML"), "html.parser"))
            if p: data.append(p)
        except: pass
    return data

def save_xlsx(leads, path):
    if not leads: return
    clean = [{k:v for k,v in l.items() if not k.startswith("_")} for l in leads if is_valid_website(l.get("Website",""))]
    if not clean: return
    for i,l in enumerate(clean,1): l["#"] = i
    df = pd.DataFrame(clean); df.to_excel(path, index=False)
    try:
        wb = load_workbook(path); ws = wb.active
        dv = DataValidation(type="list", formula1='"Not Contacted,Contacted,Interested,Not Interested,Closed Won,Closed Lost"')
        ws.add_data_validation(dv)
        for c in ws[1]:
            if c.value == "Status":
                for r in range(2, ws.max_row+1):
                    cell = ws.cell(r, c.column)
                    if not cell.value: cell.value = "Not Contacted"
                    dv.add(cell)
        wb.save(path)
    except: pass

def scrape(term, location, existing_ids):
    url = f"https://www.yellowpages.com/{location}/{term}"
    outfile = os.path.join(OUTPUT_DIR, f"yp_{NICHE_KEY}_{location}_{term}.xlsx")
    existing = []
    if os.path.exists(outfile):
        try:
            df = pd.read_excel(outfile); existing = df.to_dict('records')
            existing = [l for l in existing if is_valid_website(l.get("Website",""))]
            for l in existing: l["_id"] = gen_id(l.get("Company Name",""), l.get("Phone",""))
        except: pass
    print(f"\n{'='*60}\n{NICHE_LABEL}: {term} @ {location}\n{'='*60}")
    print(f"Filter: Only businesses with real websites (no blank/localsearch/yellowpages)")
    driver = create_driver(); leads = list(existing); local_ids = {l["_id"] for l in leads}; new_ct = 0; blocks = 0
    try:
        for pg in range(1, MAX_PAGES+1):
            pg_url = url if pg==1 else f"{url}?page={pg}"
            print(f"[Page {pg}] {pg_url}")
            try: driver.get(pg_url); time.sleep(2)
            except: continue
            listings = get_page_listings(driver)
            if not listings: print("  No results with valid websites"); break
            new_lst = [l for l in listings if l["_id"] not in existing_ids and l["_id"] not in local_ids]
            for l in new_lst: local_ids.add(l["_id"])
            print(f"  Found {len(listings)} with real websites, {len(new_lst)} new")
            if not new_lst: continue
            if FETCH_EMAILS:
                for i,l in enumerate(new_lst):
                    print(f"  [{i+1}/{len(new_lst)}] {l['Company Name'][:35]:35}", end="", flush=True)
                    em = get_email(driver, l["Source"], l.get("Website",""))
                    if em == "__BLOCKED__": print(" BLOCKED"); blocks += 1
                    elif em: l["Email"] = em; print(f" -> {em}")
                    else: print(" (no email)")
                    if blocks >= 5:
                        print("  Restarting browser..."); driver.quit(); time.sleep(5); driver = create_driver(); blocks = 0
            leads.extend(new_lst); new_ct += len(new_lst)
            save_xlsx(leads, outfile); print(f"  Saved {len(leads)} to {outfile}")
            if pg < MAX_PAGES:
                print("  Restarting browser..."); driver.quit(); time.sleep(3); driver = create_driver()
                time.sleep(random.uniform(PAGE_DELAY, PAGE_DELAY+5))
    except Exception as e: print(f"Error: {e}")
    finally:
        try: driver.quit()
        except: pass
    print(f"Done: {new_ct} new leads (all with real websites), {len(leads)} total")
    return leads, new_ct

# ============== RUN ==============
ensure_dir()
all_ids = load_all_ids()
print(f"Loaded {len(all_ids)} existing IDs")
print(f"NOTE: Only scraping businesses with real websites (filtering out blank/localsearch/yellowpages URLs)\n")

if RUN_ALL:
    total = 0
    for ti, term in enumerate(SEARCH_TERMS):
        for li, loc in enumerate(LOCATIONS):
            _, n = scrape(term, loc, all_ids)
            total += n; all_ids = load_all_ids()
            time.sleep(random.uniform(15, 30))
    print(f"\n{'='*60}\nALL DONE! {total} new leads (all with real websites)\n{'='*60}")
else:
    scrape(SEARCH_TERMS[CURRENT_TERM_INDEX], LOCATIONS[CURRENT_LOCATION_INDEX], all_ids)


Loaded 0 existing IDs
NOTE: Only scraping businesses with real websites (filtering out blank/localsearch/yellowpages URLs)


Promotional Products/Print: promotional-products @ queens-ny
Filter: Only businesses with real websites (no blank/localsearch/yellowpages)
[Page 1] https://www.yellowpages.com/queens-ny/promotional-products
  Found 9 with real websites, 9 new
  [1/9] PromosOnline                        -> helpdesk@promosonline.com
  [2/9] Nextstop Media, LLC                 (no email)
  [3/9] Nextstop Media, LLC                 (no email)
  [4/9] Brooklyn Made Corp.                 -> info@brooklynmadeco.com
  [5/9] HPD SIGNS                           (no email)
  [6/9] Minuteman Press                     (no email)
  [7/9] Ggpro Graphics                      -> SALES@GGPROGRAPHICS.COM
  [8/9] Deen Sisters                        -> SISTERSBKLYN@GMAIL.COM
  [9/9] Vida Signs                          -> vidasigns@gmail.com
  Saved 9 to exports_promo/yp_promo_queens-ny_promotional-pr

KeyboardInterrupt: 